<a href="https://colab.research.google.com/github/Hira63S/Project-2-Water-Pump/blob/master/DS_Project2_Water_Pumps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tanzanian Water Pumps#



In [72]:
!pip install kaggle

In [11]:
!pip install category_encoders

     |████████████████████████████████| 92kB 3.4MB/s 


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.exceptions import DataConversionWarning
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import category_encoders as ce
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

In [0]:
url1 = 'https://raw.githubusercontent.com/Hira63S/Project-2-Water-Pump/master/train_features.csv'

In [119]:
X_train = pd.read_csv(url1)
X_train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,3/14/2011,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,3/6/2013,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2/25/2013,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,1/28/2013,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,7/13/2011,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [0]:
url2 = 'https://raw.githubusercontent.com/Hira63S/Project-2-Water-Pump/master/train_labels.csv'

In [121]:
y_train = pd.read_csv(url2)
y_train.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


Drop id feature in y_train and it's all set for test train split

In [0]:
y_train = y_train.drop(columns = 'id')

In [0]:
X_train = X_train.drop(columns = 'id')

In [129]:
X_train.describe(include='number')

,longitude,latitude,region_code,district_code,population,construction_year
count,47520.000000,4.752000e+04,47520.000000,47520.000000,47520.000000,47520.000000
mean,34.091316,-5.705002e+00,15.326515,5.639310,179.528283,1303.353199
std,6.538403,2.943503e+00,17.618798,9.661285,472.772997,950.763878
min,0.000000,-1.164944e+01,1.000000,0.000000,0.000000,0.000000
25%,33.084320,-8.532465e+00,5.000000,2.000000,0.000000,0.000000
50%,34.911677,-5.017697e+00,12.000000,3.000000,25.000000,1986.000000
75%,37.180585,-3.326464e+00,17.000000,5.000000,213.000000,2004.000000
max,40.345193,-2.000000e-08,99.000000,80.000000,30500.000000,2013.000000


Let's start with just the numerics. Let's get rid of unnecessary columns, then split into train and validation sets and then move on to using just the numerics.

In [131]:
X_train.head(2)

,longitude,latitude,region,region_code,district_code,population,recorded_by,scheme_management,permit,construction_year,extraction_type,payment,water_quality,quantity,source,source_class
3607,35.426020,-4.227446,Manyara,21,1,160,GeoData Consultants Ltd,Water Board,True,1998,gravity,pay per bucket,soft,insufficient,spring,groundwater
50870,35.510074,-5.724555,Dodoma,1,6,0,GeoData Consultants Ltd,VWC,True,0,india mark ii,never pay,soft,enough,shallow well,groundwater


Dropping all the unnecessary columns in X_trian

In [0]:
X_train = X_train.drop(columns=['amount_tsh', 'date_recorded', 'funder', 'installer',
                               'gps_height', 'wpt_name', 'num_private','basin','subvillage',
                               'lga', 'ward', 'public_meeting', 'scheme_name',
                               'extraction_type_group', 'extraction_type_class', 'management',
                               'management_group', 'payment_type', 'quality_group',
                               'source_type', 'quantity_group', 'waterpoint_type',
                               'waterpoint_type_group'])

In [0]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42)

In [127]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((47520, 16), (11880, 16), (47520, 1), (11880, 1))

In [128]:
X_train.head()

,longitude,latitude,region,region_code,district_code,population,recorded_by,scheme_management,permit,construction_year,extraction_type,payment,water_quality,quantity,source,source_class
3607,35.426020,-4.227446,Manyara,21,1,160,GeoData Consultants Ltd,Water Board,True,1998,gravity,pay per bucket,soft,insufficient,spring,groundwater
50870,35.510074,-5.724555,Dodoma,1,6,0,GeoData Consultants Ltd,VWC,True,0,india mark ii,never pay,soft,enough,shallow well,groundwater
20413,32.499866,-9.081222,Mbeya,12,6,0,GeoData Consultants Ltd,VWC,False,0,other,never pay,soft,enough,shallow well,groundwater
52806,34.060484,-8.830208,Mbeya,12,7,0,GeoData Consultants Ltd,VWC,True,0,gravity,pay monthly,soft,insufficient,river,surface
50091,37.032690,-6.040787,Morogoro,5,1,120,GeoData Consultants Ltd,VWC,True,1997,other,pay when scheme fails,salty,enough,shallow well,groundwater


looking good so far when it comes to shape and columsn as well.
now, check for nulls

In [133]:
X_train.isnull().sum()

longitude               0
latitude                0
region                  0
region_code             0
district_code           0
population              0
recorded_by             0
scheme_management    3102
permit               2439
construction_year       0
extraction_type         0
payment                 0
water_quality           0
quantity                0
source                  0
source_class            0
dtype: int64

so some categorical have nulls but we are first going to use only numerical so let's ignore those.

In [0]:
X_train_numeric = X_train.select_dtypes('number')
X_val_numeric = X_val.select_dtypes('number')

In [135]:
X_train_numeric.shape, X_val_numeric.shape, y_train.shape, y_val.shape

((47520, 6), (11880, 6), (47520, 1), (11880, 1))

don't need to encode the numerical columns for X_train so I guess I can move on to make my model. 

let's get a majority class baseline

In [0]:
target = 'status_group'

y_train = y_train[target]

In [0]:
y_val = y_val[target]

In [138]:
y_train.head(), y_val.head()

(3607         functional
 50870        functional
 20413    non functional
 52806    non functional
 50091    non functional
 Name: status_group, dtype: object, 2980     non functional
 5246         functional
 22659        functional
 39888    non functional
 13361        functional
 Name: status_group, dtype: object)

In [139]:
y_train.value_counts(normalize=True)

functional                 0.542971
non functional             0.384091
functional needs repair    0.072938
Name: status_group, dtype: float64

In [0]:
majority_class = y_train.mode()[0]
y_pred = [majority_class] * len(y_val)

In [0]:
y_pred

In [142]:
from sklearn.metrics import accuracy_score

accuracy_score(y_val, y_pred)

0.5435185185185185

roughly the same as the percentage we got from above, maybe because we split y_val...?

can't use roc_auc for multiclass labels, let's just go on with accuracy score

I am just using the accuracy score now.

In [143]:
param_distributions = {
    'n_estimators': [100,200],
    'max_depth': [2,5]
    
}

search = RandomizedSearchCV(
    estimator = XGBClassifier(n_jobs=-1, random_state=42),
    param_distributions = param_distributions, #find active hyperparameteres
    n_iter = 50,
    scoring = 'accuracy',
    n_jobs = -1,
    cv = 3,
    verbose = 10,
    return_train_score=True
)

search.fit(X_train_numeric, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 4 is smaller than n_iter=50. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   17.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   45.1s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  1.3min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bytree=1, gamma=0,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=3, min_child_weight=1,
                                           missing=None, n_estimators=100,
                                           n_jobs=-1, nthread=None,
                                           objective='binary:logistic',
                                           random_state=42, reg_alpha=0,
                                           reg_lambda=1, scale_pos_weight=1,
                                           seed=None, silent=True,
                                           subsample=1),
                   iid='warn', n_iter=50, n_jobs=-1,
                   param_distri

In [144]:
print('Cross-Validation accuracy score:', search.best_score_)

Cross-Validation accuracy score: 0.661952861952862


So, we have score from our basic X_train dataset. Now, let's apply this to our X_val dataset to get another accuracy score.

In [145]:
X_val.head()

,longitude,latitude,region,region_code,district_code,population,recorded_by,scheme_management,permit,construction_year,extraction_type,payment,water_quality,quantity,source,source_class
2980,31.985658,-3.596360,Shinyanga,17,5,0,GeoData Consultants Ltd,WUG,True,0,other,unknown,soft,dry,shallow well,groundwater
5246,32.832815,-4.944937,Tabora,14,6,0,GeoData Consultants Ltd,VWC,True,0,india mark ii,never pay,milky,insufficient,shallow well,groundwater
22659,35.488289,-4.242048,Manyara,21,1,148,GeoData Consultants Ltd,Water Board,True,2008,gravity,pay per bucket,soft,insufficient,spring,groundwater
39888,33.140828,-9.059386,Mbeya,12,6,0,GeoData Consultants Ltd,VWC,False,0,nira/tanira,never pay,soft,seasonal,shallow well,groundwater
13361,34.217077,-4.430529,Singida,13,1,235,GeoData Consultants Ltd,WUA,True,2011,mono,pay per bucket,soft,enough,machine dbh,groundwater


In [0]:
#best = search.best_estimator_
#y_pred = best.predict_proba(X_val_numeric.values)[:,1]
#print('validation roc_Auc:', accuracy_score(y_val, y_pred))

In [146]:
results = pd.DataFrame(search.cv_results_)
results.sort_values(by='rank_test_score').head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
3,18.283689,2.176876,0.700466,0.071557,200,5,"{'n_estimators': 200, 'max_depth': 5}",0.663089,0.660290,0.662479,0.661953,0.001202,1,0.703842,0.704482,0.706859,0.705061,0.001298
2,10.179767,0.091379,0.384598,0.003814,100,5,"{'n_estimators': 100, 'max_depth': 5}",0.652926,0.649811,0.651114,0.651284,0.001277,2,0.679977,0.676768,0.676525,0.677757,0.001573
1,9.193288,0.034635,0.346229,0.002653,200,2,"{'n_estimators': 200, 'max_depth': 2}",0.622877,0.624937,0.624471,0.624095,0.000882,3,0.627072,0.633523,0.633534,0.631376,0.003044
0,4.654798,0.021950,0.189078,0.002819,100,2,"{'n_estimators': 100, 'max_depth': 2}",0.616943,0.616035,0.614117,0.615699,0.001178,4,0.618264,0.621938,0.621603,0.620602,0.001658


In [0]:
url = 'https://raw.githubusercontent.com/Hira63S/Project-2-Water-Pump/master/test_features.csv'

In [0]:
X_test =pd.read_csv(url)

In [205]:
X_test.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2/4/2013,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,Internal,Magoma,Manyara,21,3,Mbulu,Bashay,321,True,GeoData Consultants Ltd,Parastatal,NaN,True,2012,other,other,other,parastatal,parastatal,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2/4/2013,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,Pangani,Kimnyak,Arusha,2,2,Arusha Rural,Kimnyaki,300,True,GeoData Consultants Ltd,VWC,TPRI pipe line,True,2000,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2/1/2013,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,Internal,Msatu,Singida,13,2,Singida Rural,Puma,500,True,GeoData Consultants Ltd,VWC,P,NaN,2010,other,other,other,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,1/22/2013,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,Ruvuma / Southern Coast,Kipindimbi,Lindi,80,43,Liwale,Mkutano,250,NaN,GeoData Consultants Ltd,VWC,NaN,True,1987,other,other,other,vwc,user-group,unknown,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,3/27/2013,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,Ruvuma / Southern Coast,Losonga,Ruvuma,10,3,Mbinga,Mbinga Urban,60,NaN,GeoData Consultants Ltd,Water Board,BRUDER,True,2000,gravity,gravity,gravity,water board,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


In [206]:
X_test_numeric = X_test.select.dtypes(numeric)

['id',
 'amount_tsh',
 'date_recorded',
 'funder',
 'gps_height',
 'installer',
 'longitude',
 'latitude',
 'wpt_name',
 'num_private',
 'basin',
 'subvillage',
 'region',
 'region_code',
 'district_code',
 'lga',
 'ward',
 'population',
 'public_meeting',
 'recorded_by',
 'scheme_management',
 'scheme_name',
 'permit',
 'construction_year',
 'extraction_type',
 'extraction_type_group',
 'extraction_type_class',
 'management',
 'management_group',
 'payment',
 'payment_type',
 'water_quality',
 'quality_group',
 'quantity',
 'quantity_group',
 'source',
 'source_type',
 'source_class',
 'waterpoint_type',
 'waterpoint_type_group']

In [0]:
X_test_numeric = X_test.drop(columns=['id', 'amount_tsh', 'date_recorded', 'funder', 'installer',
                               'gps_height', 'wpt_name', 'num_private','basin','subvillage',
                               'lga', 'ward', 'public_meeting', 'scheme_name',
                               'extraction_type_group', 'extraction_type_class', 'management',
                               'management_group', 'payment_type', 'quality_group',
                               'source_type', 'quantity_group', 'waterpoint_type',
                               'waterpoint_type_group'])

In [197]:
X_test.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2/4/2013,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,Internal,Magoma,Manyara,21,3,Mbulu,Bashay,321,True,GeoData Consultants Ltd,Parastatal,NaN,True,2012,other,other,other,parastatal,parastatal,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2/4/2013,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,Pangani,Kimnyak,Arusha,2,2,Arusha Rural,Kimnyaki,300,True,GeoData Consultants Ltd,VWC,TPRI pipe line,True,2000,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2/1/2013,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,Internal,Msatu,Singida,13,2,Singida Rural,Puma,500,True,GeoData Consultants Ltd,VWC,P,NaN,2010,other,other,other,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,1/22/2013,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,Ruvuma / Southern Coast,Kipindimbi,Lindi,80,43,Liwale,Mkutano,250,NaN,GeoData Consultants Ltd,VWC,NaN,True,1987,other,other,other,vwc,user-group,unknown,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,3/27/2013,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,Ruvuma / Southern Coast,Losonga,Ruvuma,10,3,Mbinga,Mbinga Urban,60,NaN,GeoData Consultants Ltd,Water Board,BRUDER,True,2000,gravity,gravity,gravity,water board,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


In [211]:
X_test_numeric.shape, X_train.shape

((14358, 16), (47520, 16))

In [212]:
X_train_numeric.head()

,longitude,latitude,region_code,district_code,population,construction_year
3607,35.426020,-4.227446,21,1,160,1998
50870,35.510074,-5.724555,1,6,0,0
20413,32.499866,-9.081222,12,6,0,0
52806,34.060484,-8.830208,12,7,0,0
50091,37.032690,-6.040787,5,1,120,1997


In [0]:
X_test_numeric = X_test_numeric.drop(columns = ['recorded_by', 'scheme_management', 'permit',
                                    'extraction_type', 'payment', 'water_quality', 'quantity',
                                    'source', 'source_class'])

In [0]:
X_test_numeric = X_test_numeric.drop(columns = ['region'])

In [218]:
X_test_numeric.head()

,longitude,latitude,region_code,district_code,population,construction_year
0,35.290799,-4.059696,21,3,321,2012
1,36.656709,-3.309214,2,2,300,2000
2,34.767863,-5.004344,13,2,500,2010
3,38.058046,-9.418672,80,43,250,1987
4,35.006123,-10.950412,10,3,60,2000


In [161]:
X_train_numeric.shape

(47520, 6)

In [185]:
X_train_numeric.head()

,longitude,latitude,region_code,district_code,population,construction_year
3607,35.426020,-4.227446,21,1,160,1998
50870,35.510074,-5.724555,1,6,0,0
20413,32.499866,-9.081222,12,6,0,0
52806,34.060484,-8.830208,12,7,0,0
50091,37.032690,-6.040787,5,1,120,1997


In [188]:
X_test_numeric.head()

,longitude,latitude,region_code,district_code,population,construction_year
0,35.290799,-4.059696,21,3,321,2012
1,36.656709,-3.309214,2,2,300,2000
2,34.767863,-5.004344,13,2,500,2010
3,38.058046,-9.418672,80,43,250,1987
4,35.006123,-10.950412,10,3,60,2000


In [189]:
X_test.head()

,date_recorded,funder,installer,longitude,latitude,wpt_name,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,2/4/2013,Dmdd,DMDD,35.290799,-4.059696,Dinamu Secondary School,Internal,Magoma,Manyara,21,3,Mbulu,Bashay,321,True,GeoData Consultants Ltd,Parastatal,NaN,True,2012,other,other,other,parastatal,parastatal,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,2/4/2013,Government Of Tanzania,DWE,36.656709,-3.309214,Kimnyak,Pangani,Kimnyak,Arusha,2,2,Arusha Rural,Kimnyaki,300,True,GeoData Consultants Ltd,VWC,TPRI pipe line,True,2000,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,2/1/2013,NaN,NaN,34.767863,-5.004344,Puma Secondary,Internal,Msatu,Singida,13,2,Singida Rural,Puma,500,True,GeoData Consultants Ltd,VWC,P,NaN,2010,other,other,other,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,1/22/2013,Finn Water,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,Ruvuma / Southern Coast,Kipindimbi,Lindi,80,43,Liwale,Mkutano,250,NaN,GeoData Consultants Ltd,VWC,NaN,True,1987,other,other,other,vwc,user-group,unknown,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,3/27/2013,Bruder,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,Ruvuma / Southern Coast,Losonga,Ruvuma,10,3,Mbinga,Mbinga Urban,60,NaN,GeoData Consultants Ltd,Water Board,BRUDER,True,2000,gravity,gravity,gravity,water board,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


In [0]:
# estimator is your model or pipeline, which you've fit on X_train

# X_test is your pandas dataframe or numpy array, 
# with the same number of rows, in the same order, as test_features.csv, 
# and the same number of columns, in the same order, as X_train

y_pred = search.predict(X_test_numeric)


# Makes a dataframe with two columns, id and status_group, 
# and writes to a csv file, without the index

#filename = 'Water Pump1.csv'
#submission.to_csv(filename,index=False)
#print('Saved file:' + filename)

##sample_submission = pd.read_csv('sample_submission.csv')
#submission = sample_submission.copy()
#submission['status_group'] = y_pred
#submission.to_csv('your-submission-filename.csv', index=False)

In [0]:
submission = pd.DataFrame({'id': X_test['id'], 'status_group': list(y_pred)})

In [223]:
submission.head()


,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional


In [225]:
filename = 'first submission.csv'
submission.to_csv(filename,index=False)

print('saved file:' + filename)

saved file:first submission.csv
